In [16]:
dict1 = {1:"a", 2:"b"}
print(dict1)
s = list(dict1.items())
print(s)
for i in s:
    print(i[0], i[1])

{1: 'a', 2: 'b'}
[(1, 'a'), (2, 'b')]
1 a
2 b


In [17]:
dict1[3]= "b"
print(dict1)
dict2 = {4:"d"}

{1: 'a', 2: 'b', 3: 'b'}


In [19]:
dict1.update(dict2)
print(dict1)

{1: 'a', 2: 'b', 3: 'b', 4: 'd'}


In [1]:
import pandas as pd

data = [{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}]

formatted_data = []

for entry in data:
    formatted_entry = {}
    for key, value in entry.items():
        formatted_entry[key] = value['S']
    formatted_data.append(formatted_entry)

df = pd.DataFrame(formatted_data)

print(df)


  raw_ingestion_status inhance_injection source_system  status    table_name  \
0            Processed         Completed          alfa  Active  SampleTable1   
1            Processed        inprogress          alfa  Active  SampleTable2   
2            Processed              Fail          alfa  Active  SampleTable3   
3              Pending              Fail          alfa  Active  SampleTable4   

  file_name_time  
0    file1_time1  
1    file2_time2  
2    file3_time3  
3    file3_time4  


In [1]:
list_table = ["critical_process", "non_critical_process"]
for table_name in list_table:
    print(table_name)

critical_process
non_critical_process


In [36]:
import pandas as pd

data = [{'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Completed'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable1'}, 'file_name_time': {'S': 'file1_time1'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'inprogress'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable2'}, 'file_name_time': {'S': 'file2_time2'}}, {'raw_ingestion_status': {'S': 'Processed'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable3'}, 'file_name_time': {'S': 'file3_time3'}}, {'raw_ingestion_status': {'S': 'Pending'}, 'inhance_injection': {'S': 'Fail'}, 'source_system': {'S': 'alfa'}, 'status': {'S': 'Active'}, 'table_name': {'S': 'SampleTable4'}, 'file_name_time': {'S': 'file3_time4'}}]

# Extracting data into a list of dictionaries
formatted_data = [{key: value['S'] for key, value in entry.items()} for entry in data]

# Creating DataFrame
df = pd.DataFrame(formatted_data)

print(df)


  raw_ingestion_status inhance_injection source_system  status    table_name  \
0            Processed         Completed          alfa  Active  SampleTable1   
1            Processed        inprogress          alfa  Active  SampleTable2   
2            Processed              Fail          alfa  Active  SampleTable3   
3              Pending              Fail          alfa  Active  SampleTable4   

  file_name_time  
0    file1_time1  
1    file2_time2  
2    file3_time3  
3    file3_time4  


In [1]:
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

class DynamoDBHelper:
    def __init__(self, region_name='us-east-1'):
        self.dynamodb = boto3.client('dynamodb', region_name=region_name)
        self.spark = self.get_spark_session()

    def get_spark_session(self):
        return SparkSession.builder \
            .appName("YourAppName") \
            .getOrCreate()

    def get_entries(self, table_name):
        try:
            response = self.dynamodb.scan(
                TableName=table_name
            )
            items = response['Items']
            return items
        except Exception as e:
            print("Error:", e)
            return None

    def formatted_df_spark(self, table_name):
        output_res_items = self.get_entries(table_name)
        formatted_data = [{key: value['S'] for key, value in entry.items()} for entry in output_res_items]
        df = self.spark.createDataFrame(formatted_data)
        return df

    # Other methods can be added here as needed

# Example usage:
if __name__ == "__main__":
    dynamodb_helper = DynamoDBHelper()
    
    table_names = ["critical_process", "non_critical_process"]
    
    for table_name in table_names:
        df = dynamodb_helper.formatted_df_spark(table_name)
        print(f"DataFrame for {table_name}:")
        df.show()


24/03/24 12:39:21 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.25 instead (on interface wlp0s20f3)
24/03/24 12:39:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/24 12:39:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Error: An error occurred (ResourceNotFoundException) when calling the Scan operation: Requested resource not found


TypeError: 'NoneType' object is not iterable